In [209]:
import pandas as pd
import json
import copy
from faker import Faker
from datetime import datetime, timedelta
import re
import random

Template Json Pagos Workflow

In [210]:
pagostemp = {
    "infoGeneral":{
      "compania":"1 - Empresa ABC",
      "bancoCorresponsal": "Bancolombia",
      "estado":"Cargado",
      "fechaCarga":"17-03-2023"
     },
     "infoOrdenante": {
      "numCuenta": {
        "id": "cta123456",
        "tipo": "AHO-Ahorros"
      },
      "nombre": " Filial Empresa ABC LTDA",
      "direccion": {
          "numero": "Cl 14 52 A 20",
          "ciudad": "Medellin",
          "pais": "CO"
       }
      },
      "infoBeneficiario": {
        "numCuenta": {
          "id": "cta123456",
          "tipo": "AHO-Ahorros"
        },
        "nombre": " Carbones LTDA",
        "direccion": {
            "numero": "645 - Bulevar Sunset",
            "ciudad": "Singapore",
            "pais": "SG"
         }
      },
     "infoPagos": 
      {
        "Referencias": {
          "id": "pacs8bizmsgidr01",
          "endToEndId": "pacs008EndToEndId-111",
          "uetr": "f1636d6e-1ed0-4b56-a7be-5706eb62ba9f"
        },
        "Monto": {
          "moneda": "USD",
          "valor": "50.00"
        },
        "fechaPago": "17-03-2023",
        "cargos": "Compartidos",
        "infoAdicional": "Facturas No: 999999999"
      }
  }

In [211]:
arreglo = []
faker = Faker('es_ES')
fecha_actual = datetime.now()

Leer archivo de Datos

In [212]:
excel_file = '/Users/wilfredysantamariaruiz/Downloads/Data Dummy DEMO EMH For Corporate.xlsx'

In [213]:
df = pd.read_excel(excel_file)

In [214]:
df.head(2)

,Empresa,Negocio,Banco,cuenta,Divisa,Depto
0,Nestle,Purina,BBVA,00123456789123456789,USD,Financiero
1,Nestle,Purina,BBVA,00123456789123456790,USD,Tesoreria


In [215]:
def myclean(cadena):
    # Utilizar una expresión regular para eliminar caracteres que no sean números o letras
    cadena_limpia = re.sub(r'[^a-zA-Z0-9\s]', '', cadena)
    return cadena_limpia


In [216]:
def randoncta():
    numero_cuenta = ""
    # Generar los primeros 8 dígitos del número de cuenta
    for _ in range(8):
        digito = random.randrange(10)  # Generar un dígito aleatorio del 0 al 9
        numero_cuenta += str(digito)
    
    # Generar el último dígito verificador del número de cuenta
    digito_verificador = random.randrange(10)  # Generar un dígito aleatorio del 0 al 9
    numero_cuenta += str(digito_verificador)
    return numero_cuenta

Modificar Plantilla de pagos

In [217]:
i=1
for indice in df.index:
    nombre_aleatorio = faker.name()
    faker.address()
    pago_copia = copy.deepcopy(pagostemp)
    nueva_fecha = fecha_actual + timedelta(days=i)
    fecha_formateada = nueva_fecha.strftime("%d-%m-%Y")
    pago_copia['infoGeneral']['compania']= "-".join([df.loc[indice, 'Empresa'],df.loc[indice, 'Negocio']])
    pago_copia['infoGeneral']['bancoCorresponsal']=df.loc[indice, 'Banco']
    pago_copia['infoGeneral']['fechaCarga']=fecha_formateada
    #info ordenante
    pago_copia['infoOrdenante']['numCuenta']['id']=df.loc[indice, 'cuenta']
    pago_copia['infoOrdenante']['nombre']="-".join([df.loc[indice, 'Empresa'],df.loc[indice, 'Negocio']])
    #pago_copia['infoOrdenante']['direccion']['numero']= myclean(faker.address()) 
    #info Beneficiario
    pago_copia['infoBeneficiario']['numCuenta']['id']=randoncta()
    pago_copia['infoBeneficiario']['nombre']=myclean(faker.name())
    pago_copia['infoBeneficiario']['direccion']['numero']= myclean(faker.address()) 
    pago_copia['infoBeneficiario']['direccion']['ciudad']= myclean(faker.city()) 
    pago_copia['infoBeneficiario']['direccion']['pais']=faker.country_code()
    #info de pagos
    pago_copia['infoPagos']['Referencias']['id']="pacs"+randoncta()
    pago_copia['infoPagos']['Referencias']['endToEndId']="pacs008EndToEndId-"+ str(random.randint(100, 500))
    pago_copia['infoPagos']['Monto']['moneda']=df.loc[indice, 'Divisa']
    pago_copia['infoPagos']['Monto']['valor']=round(random.uniform(1000, 10000), 2)
    pago_copia['infoPagos']['fechaPago']=fecha_formateada
    pago_copia['infoPagos']['infoAdicional']= "Facturas No: "+str(random.randint(100, 5000)) 
    arreglo.append(pago_copia)
    i=i+1
    
    

Generar Archivo JSON-WorkFlow Pagos

In [218]:
json_file = '/Users/wilfredysantamariaruiz/Downloads/pagos-workflow.json'

In [219]:
# Escribe los datos en un archivo JSON
def mywrite(json_file,arreglo):
    with open(json_file, 'w') as file:
      json.dump(arreglo, file, indent=4)

In [220]:
mywrite(json_file,arreglo)

Template Json Movimientos

In [221]:
movtemp={
    "infoGeneral":{
      "compania":"1 - Empresa ABC",
      "bancoCorresponsal": "Bancolombia",
      "estado":"Cargado",
      "fechaCarga":"17-03-2023",
      "numCuenta": {
        "id": "cta123456",
        "tipo": "AHO-Ahorros"
      }
     },
     "infoMovimientos": [
      {
        "Referencias": {
          "id": "pacs8bizmsgidr01",
          "endToEndId": "pacs008EndToEndId-111",
          "uetr": "f1636d6e-1ed0-4b56-a7be-5706eb62ba9f"
        },
        "Monto": {
          "moneda": "USD",
          "valor": "50.00"
        },
        "fechaValor": "17-03-2023",
        "naturaleza": "Debito",
        "infoAdicional": "Pago proveedor telas"
      },
      {
        "Referencias": {
          "id": "pacs8bizmsgidr02",
          "endToEndId": "pacs008EndToEndId-112",
          "uetr": "f1636d6e-1ed0-4b56-a7be-5706eb62ba9f"
        },
        "Monto": {
          "moneda": "USD",
          "valor": "110.00"
        },
        "fechaValor": "17-03-2023",
        "naturaleza": "Debito",
        "infoAdicional": "Transferencias bancaria pago impuestos"
      }
    ]
  }

In [222]:
arreglomov = []
naturaleza = ['Debito', 'Credito']
conceptos=['Transferencias electronicas u otras formas','Cargos o comisiones','Intereses','cheques emitidos','sumando los intereses ganados']


Modificar Plantilla de Movimientos

In [223]:
i=1
for indice in df.index:
    mov_copia = copy.deepcopy(movtemp)
    nueva_fecha = fecha_actual + timedelta(days=i)
    fecha_formateada = nueva_fecha.strftime("%d-%m-%Y")
    mov_copia['infoGeneral']['compania']= "-".join([df.loc[indice, 'Empresa'],df.loc[indice, 'Negocio']])
    mov_copia['infoGeneral']['bancoCorresponsal']=df.loc[indice, 'Banco']
    mov_copia['infoGeneral']['fechaCarga']=fecha_formateada
    mov_copia['infoGeneral']['numCuenta']['id']=df.loc[indice, 'cuenta']
    #info movimientos
    mov_copia['infoMovimientos'][0]['Referencias']['id']="pacs"+randoncta()
    mov_copia['infoMovimientos'][0]['Referencias']['endToEndId']="pacs008EndToEndId-"+ str(random.randint(100, 500))
    mov_copia['infoMovimientos'][0]['Monto']['moneda']=df.loc[indice, 'Divisa']
    mov_copia['infoMovimientos'][0]['Monto']['valor']=round(random.uniform(1000, 10000), 2)
    mov_copia['infoMovimientos'][0]['fechaValor']=fecha_formateada
    mov_copia['infoMovimientos'][0]['naturaleza']=random.choice(naturaleza)
    mov_copia['infoMovimientos'][0]['infoAdicional']=random.choice(conceptos) 
    #bloque 2
    mov_copia['infoMovimientos'][1]['Referencias']['id']="pacs"+randoncta()
    mov_copia['infoMovimientos'][1]['Referencias']['endToEndId']="pacs008EndToEndId-"+ str(random.randint(100, 500))
    mov_copia['infoMovimientos'][1]['Monto']['moneda']=df.loc[indice, 'Divisa']
    mov_copia['infoMovimientos'][1]['Monto']['valor']=round(random.uniform(1000, 10000), 2)
    mov_copia['infoMovimientos'][1]['fechaValor']=fecha_formateada
    mov_copia['infoMovimientos'][1]['naturaleza']=random.choice(naturaleza)
    mov_copia['infoMovimientos'][1]['infoAdicional']=random.choice(conceptos) 
    arreglomov.append(mov_copia)
    i=i+1

In [224]:
json_filemov = '/Users/wilfredysantamariaruiz/Downloads/Extractos-Consulta movimientos.json'
mywrite(json_filemov,arreglomov)

Template Json Posicion-Saldos

In [225]:
postemp= {
    "infoGeneral":{
      "compania":"1 - Empresa ABC",
      "bancoCorresponsal": "Bancolombia",
      "estado":"Cargado",
      "fechaCarga":"17-03-2023",
      "numCuenta": {
        "id": "cta123456",
        "tipo": "AHO-Ahorros"
      },
      "fechaInicio":"12-03-2023",
      "fechaFin":"12-03-2023"
     },
     "infobalance":
     [
      {
        "tipo":  "Saldo Inicial",
        "Monto": {
          "moneda": "USD",
          "valor": "50.00"
        },
         "naturaleza": "DEBITO"
      },
      {
        "tipo":  "Saldo Final",
        "Monto": {
          "moneda": "USD",
          "valor": "50.00"
        },
        "naturaleza": "DEBITO"
      },
      {
        "tipo":  "Saldo Disponible",
        "Monto": {
          "moneda": "USD",
          "valor": "50.00"
        },
         "naturaleza": "DEBITO"
      },
      {
        "tipo":  "Saldo en Canje",
        "Monto": {
          "moneda": "USD",
          "valor": "10.00"
        },
         "naturaleza": "DEBITO"
      }
    ]
  }

In [226]:
arreglopos = []

Modificar Plantilla de Saldos

In [227]:
i=1
for indice in df.index:
    pos_copia = copy.deepcopy(postemp)
    nueva_fecha = fecha_actual + timedelta(days=i)
    fecha_formateada = nueva_fecha.strftime("%d-%m-%Y")
    pos_copia['infoGeneral']['compania']= "-".join([df.loc[indice, 'Empresa'],df.loc[indice, 'Negocio']])
    pos_copia['infoGeneral']['bancoCorresponsal']=df.loc[indice, 'Banco']
    pos_copia['infoGeneral']['fechaCarga']=fecha_formateada
    pos_copia['infoGeneral']['numCuenta']['id']=df.loc[indice, 'cuenta']
    pos_copia['infoGeneral']['fechaInicio']=fecha_formateada
    pos_copia['infoGeneral']['fechaFin']=fecha_formateada
    #info saldos
    pos_copia['infobalance'][0]['Monto']['moneda']=df.loc[indice, 'Divisa']
    pos_copia['infobalance'][0]['Monto']['valor']=round(random.uniform(10000, 40000), 2)
    pos_copia['infobalance'][0]['naturaleza']=random.choice(naturaleza)
    #bloque 2
    pos_copia['infobalance'][1]['Monto']['moneda']=df.loc[indice, 'Divisa']
    pos_copia['infobalance'][1]['Monto']['valor']=round(random.uniform(10000, 40000), 2)
    pos_copia['infobalance'][1]['naturaleza']=random.choice(naturaleza)
    #bloque 3
    pos_copia['infobalance'][2]['Monto']['moneda']=df.loc[indice, 'Divisa']
    pos_copia['infobalance'][2]['Monto']['valor']=round(random.uniform(5000, 10000), 2)
    pos_copia['infobalance'][2]['naturaleza']=random.choice(naturaleza)
    #bloque 4
    pos_copia['infobalance'][3]['Monto']['moneda']=df.loc[indice, 'Divisa']
    pos_copia['infobalance'][3]['Monto']['valor']=round(random.uniform(5000, 10000), 2)
    pos_copia['infobalance'][3]['naturaleza']=random.choice(naturaleza)
    arreglopos.append(pos_copia)
    i=i+1

In [228]:
json_filepos = '/Users/wilfredysantamariaruiz/Downloads/Extractos-Posicion Bancaria.json'
mywrite(json_filepos,arreglopos)